In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features.build_features import obtener_diccionario_mai

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
codigos_mai = obtener_diccionario_mai()

## 1. Calculo de tiempo en pabellón para cada uno de los procedimientos

En este cuadernillo se quiere responder a las siguientes preguntas:

1. ¿Cuánto es el tiempo que se demora cada procedimiento en pabellón en el INT?
2. ¿Cómo es el perfil del tiempo en pabellón para cada procedimiento por separado? ¿Este perfil
cambia a lo largo de los años?


### 1.1 Calculo de control interno de la duración
 
Una de las primeras interrogantes que se quiere responder es sobre la columna de duración
actualmente incluida en la base de datos. En primer lugar, se quiere saber si es que la duración
esta correctamente calculada.

In [ ]:
df = pd.read_csv("../data/interim/df_procesada.csv", sep=";", encoding="latin-1")
df = df.merge(codigos_mai, left_on="codigo_i", right_on="CODIGO", how="left")
df["fecha"] = pd.to_datetime(df["fecha"])

COLS_HORAS = list(df.columns[df.columns.str.startswith("h_")])
df[COLS_HORAS] = df[COLS_HORAS].apply(lambda x: pd.to_datetime(x, format="%H:%M:%S"))

salida_menor_a_entrada = df["h_salida"] < df["h_entrada"]
df.loc[salida_menor_a_entrada, "h_salida"] = df.loc[
    salida_menor_a_entrada, "h_salida"
] + pd.DateOffset(days=1)

df["duracion_calculada"] = df["h_salida"] - df["h_entrada"]
df["duracion_calculada_en_str"] = df["duracion_calculada"].astype(str).str.split(" ").str[-1]
df["duracion_calculada_horas"] = df["duracion_calculada"] / pd.Timedelta(hours=1)

In [ ]:
display(df[["duracion", "duracion_calculada_en_str"]])

De forma general se observa que las duraciones son exactamente iguales entre si en los ejemplos
mostrados en pantalla. Sin embargo, es necesario saber si es qu existe alguna diferencia.

In [ ]:
display(df[df["duracion"] != df["duracion_calculada_en_str"]].dropna(subset="duracion"))

Se puede observar que solamente 6 registros (de 15897) tuvieron una diferencia en la duración calculada.
En estos casos, se utilizará la duración calculada.

### 1.2 Obtención de duración por tipo de operación

En este análisis se quiere ver la distribución de la duración de cada operación realizada en el INT.

In [ ]:
sns.histplot(data=df, x="duracion_calculada_horas", hue="codigo_i", element="step", kde=True)

En este gráfico se puede observar la distribución de todas las operaciones realizadas en el INT,
sin distinción entre años.

In [ ]:
for operacion in df["codigo_i"].unique():
    df_operacion = df.query("codigo_i == @operacion")
    nombre_operacion = str(df_operacion["GLOSA PRESTACION"].iloc[0])[:100]
    sns.histplot(data=df_operacion, x="duracion_calculada_horas", element="step", kde=True)
    plt.title(f"{operacion} {nombre_operacion} - n = {df_operacion.shape[0]}")
    plt.show()

In [ ]:
resumen_duracion = df.groupby([df["fecha"].dt.year, "codigo_i", "GLOSA PRESTACION"], dropna=False)[
    "duracion_calculada_horas"
].describe()

In [ ]:
display(resumen_duracion)

Finalmente, se puede ver la distribución de cada operación realizada en cada año.